In [1]:
import pandas as pd
from tqdm.notebook import tqdm
from datetime import datetime
import statistics
import numpy as np

In [3]:
# This notebook simulates the trading strategy on a fixed reference price
# we therefore ignore implicit and explicit costs

# Load tick data
#tickdata = pd.read_csv("data/ETH_2021.csv")
tickdata = pd.read_csv("data/ETH_2022.csv")
tickdata = tickdata.drop(["Unnamed: 0"], axis = 1)
tickdata.head(10)

,Date,Ethereum[USD],Bitcoin[USD]
0,2022-01-01 00:00:00,3683.70,46393.0
1,2022-01-01 01:00:00,3678.20,46196.9
2,2022-01-01 02:00:00,3722.62,46695.2
3,2022-01-01 03:00:00,3731.51,46828.2
4,2022-01-01 04:00:00,3730.53,46805.2
5,2022-01-01 05:00:00,3730.67,46873.8
6,2022-01-01 06:00:00,3715.62,46790.3
7,2022-01-01 07:00:00,3735.91,47273.0
8,2022-01-01 08:00:00,3718.95,47045.7
9,2022-01-01 09:00:00,3719.46,47236.1


In [10]:
# Load funding rates
#df_funding_rates = pd.read_csv("data/fundingrates21_ETHUSDT.csv")
df_funding_rates = pd.read_csv("data/fundingrates22_ETHUSDT.csv")
#df_funding_rates = df_funding_rates.drop(["Unnamed: 0"], axis = 1)
df_funding_rates.head(10)

,timestamp,symbol,fundingInterval,fundingrates,fundingRateDaily
0,2022-01-01T04:00:00.000Z,ETHUSDT,2000-01-01T08:00:00.000Z,0.0001,0.0003
1,2022-01-01T12:00:00.000Z,ETHUSDT,2000-01-01T08:00:00.000Z,0.0001,0.0003
2,2022-01-01T20:00:00.000Z,ETHUSDT,2000-01-01T08:00:00.000Z,0.0001,0.0003
3,2022-01-02T04:00:00.000Z,ETHUSDT,2000-01-01T08:00:00.000Z,0.0001,0.0003
4,2022-01-02T12:00:00.000Z,ETHUSDT,2000-01-01T08:00:00.000Z,0.0001,0.0003
5,2022-01-02T20:00:00.000Z,ETHUSDT,2000-01-01T08:00:00.000Z,0.0001,0.0003
6,2022-01-03T04:00:00.000Z,ETHUSDT,2000-01-01T08:00:00.000Z,0.0001,0.0003
7,2022-01-03T12:00:00.000Z,ETHUSDT,2000-01-01T08:00:00.000Z,0.0001,0.0003
8,2022-01-03T20:00:00.000Z,ETHUSDT,2000-01-01T08:00:00.000Z,0.0001,0.0003
9,2022-01-04T04:00:00.000Z,ETHUSDT,2000-01-01T08:00:00.000Z,0.0001,0.0003


In [5]:
list_Date = tickdata["Date"].tolist()
list_Ethereum = tickdata["Ethereum[USD]"].tolist()
list_Bitcoin = tickdata["Bitcoin[USD]"].tolist()

In [6]:
def fee(historic_trading_volume, additional_trading_volume):
    if historic_trading_volume < 1000000:
        fee = 0.075*0.01
    elif historic_trading_volume < 5000000:
        fee = 0.05*0.01
    elif historic_trading_volume < 10000000:
        fee = 0.04*0.01
    elif historic_trading_volume < 25000000:
        fee = 0.03*0.01
    elif historic_trading_volume < 50000000:
        fee = 0.025*0.01
    elif historic_trading_volume < 100000000:
        fee = 0.024*0.01
    elif historic_trading_volume < 250000000:
        fee = 0.023*0.01
    
    return fee*additional_trading_volume

In [7]:
def compute_volatility(list_volumes):
    list_daily_return = []
    for i in range(1, len(list_volumes)):
        list_daily_return.append((list_volumes[i] - list_volumes[i-1])/list_volumes[i-1])
    volatility = statistics.stdev(list_daily_return)*np.sqrt(len(list_daily_return))
    return volatility

In [8]:
def simulation(threshold, volume):
    # Opening the Position
    ETH_USD_start = list_Ethereum[0] # spot price of Ethereum on the 01/01/2021 at 0am
    BTC_USD_start = list_Bitcoin[0] # spot price of Bitcoin on the 01/01/2021 at 0am

    list_closing_volume = []
    # constants
    multiplier = 0.00001 # if the spot price of Ethereum rises by 1$ a swap contracts decreases by 0.00001BTC

    volume_investment_start = volume # the total volume which is initially invested in USD
    volume_investment_spot_start = volume_investment_start*0.5 # half of the total volume is initially invested in USD
    volume_investment_swap_start = volume_investment_start*0.5 # half of the total volume is initially invested in USD

    # compute the number of Ethereum coins
    spot_ETH_n = volume_investment_spot_start/ETH_USD_start
    spot_ETH_USD = ETH_USD_start

    # compute the number of swap contracts
    swap_n = round(volume_investment_swap_start/(multiplier*ETH_USD_start)) # there are no fractional contracts
    swap_ETH_USD = ETH_USD_start # the ETH entry price of the swap contracts, this changes when a new position is opened
    
    funding_rates = 0.0

    j = 0 # running index for catching funding rate
    #threshold = 0.001
    #threshold = 0.0001

    number_trades = 0
    historic_trading_volume = 0
    trading_fees = 0
    for i in range(1,len(list_Ethereum)):
        ETH_USD_now = list_Ethereum[i]
        BTC_USD_now = list_Bitcoin[i]
        

        # Computing the value of the two positions at another time
        # the value of the spot position
        volume_spot = spot_ETH_n*ETH_USD_now

        # the PNL in USD for the swap position is given by
        PNL_USD = -(ETH_USD_now - swap_ETH_USD)*multiplier*swap_n
        # the value of the swap position is then given by
        volume_swap = PNL_USD + swap_ETH_USD*multiplier*swap_n

        # computation of funding rate
        datetime_str = list_Date[i][2:]
        datetime_object = datetime.strptime(datetime_str, '%y-%m-%d %H:%M:%S')
        if datetime_object.hour in [6,12,21]: # funding rate is paid at 4, 12, 20; we make 6,12,21 as we have 3h intervalls
            funding_rates = funding_rates + df_funding_rates["fundingrates"].tolist()[j]*volume_swap
            j = j+1
        
        if datetime_object.hour in [21]: # funding rate is paid at 4, 12, 20; we make 6,12,21 as we have 3h intervalls
            list_closing_volume.append(volume_spot + volume_swap + funding_rates - trading_fees)

        if abs((volume_spot - volume_swap)/(volume_spot + volume_swap)) > threshold:
            number_trades += 1
            trading_fees += fee(historic_trading_volume, 0.5*abs(volume_spot - volume_swap))
            historic_trading_volume += 0.5*abs(volume_spot - volume_swap)
            
            if volume_swap > volume_spot:
                spot_ETH_n_pre = spot_ETH_n # the number of ETH before (pre) adjustment
                spot_ETH_USD_pre = spot_ETH_USD
                swap_n_pre = swap_n
                swap_ETH_USD_pre = swap_ETH_USD

                # computation of new spot_ETH_n
                volume_to_adjust = 0.5*(volume_swap + volume_spot) - volume_spot
                n_ETH_additional = volume_to_adjust/ETH_USD_now
                spot_ETH_n = spot_ETH_n_pre + n_ETH_additional

                # computation of new spot_ETH_USD
                spot_ETH_USD = (spot_ETH_n_pre*spot_ETH_USD_pre + n_ETH_additional*ETH_USD_now)/spot_ETH_n

                # computation of new swap_ETH_USD
                swap_ETH_USD = swap_ETH_USD_pre # the entry price for ETH does not change, when the number of contracts is decreased

                # computation of new swap_n
                # PNL in USD for one single swap contract
                PNL_USD_now = -(ETH_USD_now - swap_ETH_USD_pre)*multiplier
                # value of one swap contract now
                value_swap_contract_now = multiplier*swap_ETH_USD_pre + PNL_USD_now
                # number of swap contracts that need to be reduced
                n_swap_additional = round(volume_to_adjust/value_swap_contract_now)
                # new number of swap contracts
                swap_n = swap_n_pre - n_swap_additional

            else:
                spot_ETH_n_pre = spot_ETH_n # the number of ETH before (pre) adjustment
                spot_ETH_USD_pre = spot_ETH_USD
                swap_n_pre = swap_n
                swap_ETH_USD_pre = swap_ETH_USD

                # computation of new spot_ETH_n
                volume_to_adjust = 0.5*(volume_swap + volume_spot) - volume_swap
                n_ETH_additional = (-1)*volume_to_adjust/ETH_USD_now
                spot_ETH_n = spot_ETH_n_pre + n_ETH_additional

                # computation of new spot_ETH_USD
                spot_ETH_USD = spot_ETH_USD_pre # the entry price for ETH does not change, when the number of contracts is decreased

                # computation of new swap_n
                n_swap_additional = round(volume_to_adjust/(ETH_USD_now*multiplier))
                swap_n = swap_n_pre + n_swap_additional

                # computation of new swap_ETH_USD
                swap_ETH_USD = (swap_n_pre*swap_ETH_USD + n_swap_additional*ETH_USD_now)/swap_n
                
    print(len(list_closing_volume))
    latest_ETH_price = list_Ethereum[-1]
    latest_BTC_price = list_Bitcoin[-1]
    
    volume_total = spot_ETH_n*latest_ETH_price + swap_n*multiplier*swap_ETH_USD - swap_n*multiplier*(latest_ETH_price-swap_ETH_USD) + funding_rates
    returns_made = volume_total - volume
    
    volatility = compute_volatility(list_closing_volume)

    #print("Data related to Spot Position")
    #print("Number of Ethereum coins: " + str(spot_ETH_n))
    #print("average price paid for an Ethereum coin: " + str(spot_ETH_USD))
    #print("Volume of spot position: " + str(spot_ETH_n*latest_ETH_price))
    #print()
    #print("Data related to Swap Position")
    #print("Number of swap contracts: " + str(swap_n))
    #print("Average entry price of each contract: " + str(swap_ETH_USD))
    #print("Volume of swap position: " + str(swap_n*multiplier*latest_BTC_price*swap_ETH_USD - swap_n*multiplier*latest_BTC_price*(latest_ETH_price-swap_ETH_USD)))
    #print()
    #print("Funding Rates")
    #print(funding_rates)
    
    return volume_total, returns_made, trading_fees, volatility

In [11]:
list_volume_evaluated = [50000, 100000, 250000, 500000, 750000, 1000000, 1500000, 2500000, 5000000, 7500000]
list_threshold_evaluated = [0.04, 0.03, 0.02, 0.01]

list_threshold = []
list_volume = []
list_volume_total = []
list_returns_made = []
list_fees = []
list_volatility = []
list_return_percentage = []
list_sharpe_ratio = []

for t in list_threshold_evaluated:
    for v in list_volume_evaluated:
        print(t, v)
        list_threshold.append(t)
        list_volume.append(v)
        
        volume_total, returns_made, trading_fees, volatility = simulation(t, v)
        list_return_percentage.append(round(returns_made/v,4))
        list_sharpe_ratio.append(round(returns_made/(v*volatility),2))
        
        list_volume_total.append(volume_total)
        list_returns_made.append(returns_made)
        list_fees.append(trading_fees)
        list_volatility.append(volatility)

results = pd.DataFrame({'threshold': list_threshold, 'volume_invest': list_volume, 'return': list_return_percentage, 'sharpe-ratio': list_sharpe_ratio})

0.04 50000
243
0.04 100000
243
0.04 250000
243
0.04 500000
243
0.04 750000
243
0.04 1000000
243
0.04 1500000
243
0.04 2500000
243
0.04 5000000
243
0.04 7500000
243
0.03 50000
243
0.03 100000
243
0.03 250000
243
0.03 500000
243
0.03 750000
243
0.03 1000000
243
0.03 1500000
243
0.03 2500000
243
0.03 5000000
243
0.03 7500000
243
0.02 50000
243
0.02 100000
243
0.02 250000
243
0.02 500000
243
0.02 750000
243
0.02 1000000
243
0.02 1500000
243
0.02 2500000
243
0.02 5000000
243
0.02 7500000
243
0.01 50000
243
0.01 100000
243
0.01 250000
243
0.01 500000
243
0.01 750000
243
0.01 1000000
243
0.01 1500000
243
0.01 2500000
243
0.01 5000000
243
0.01 7500000
243


In [12]:
results.head(10)

,threshold,volume_invest,return,sharpe-ratio
0,0.04,50000,-0.026,-0.18
1,0.04,100000,-0.026,-0.18
2,0.04,250000,-0.026,-0.18
3,0.04,500000,-0.026,-0.18
4,0.04,750000,-0.026,-0.18
5,0.04,1000000,-0.026,-0.18
6,0.04,1500000,-0.026,-0.18
7,0.04,2500000,-0.026,-0.18
8,0.04,5000000,-0.026,-0.18
9,0.04,7500000,-0.026,-0.18


In [14]:
results.to_csv("results_ETHUSDT_22.csv")